### 미니 프로젝트

In [44]:
import numpy as np
import pandas as pd

### CCTV 데이터 정리하기

- 데이터 파일 읽기

In [21]:
filename = '서울시CCTV설치운영현황(자치구)_년도별_211231기준.csv'
cctv = pd.read_csv(filename, skiprows=1, encoding='EUC-KR')
cctv.head()

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,"83,557","4,812","1,851","3,434","4,295","6,840","8,708","11,572","10,627","12,267","11,247","7,904"
1,종로구,"1,715",815,NaN,NaN,195,150,0,261,85,9,200,0
2,중 구,"2,447",16,114,87,77,236,240,372,386,155,361,403
3,용산구,"2,611",34,71,234,125,221,298,351,125,307,617,228
4,성동구,"3,829",163,144,208,107,325,255,967,415,490,472,283


- 결측치(Null data) 확인

In [3]:
# 2012년, 2013년에 각각 1개씩 결측치 존재
cctv.isnull().sum()             

구분          0
총계          0
2012년 이전    0
2012년       1
2013년       1
2014년       0
2015년       0
2016년       0
2017년       0
2018년       0
2019년       0
2020년       0
2021년       0
dtype: int64

- 결측치를 0으로 대체

In [22]:
# 결측치를 0으로 대체
cctv.fillna('0', inplace=True)
cctv.head()

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,"83,557","4,812","1,851","3,434","4,295","6,840","8,708","11,572","10,627","12,267","11,247","7,904"
1,종로구,"1,715",815,0,0,195,150,0,261,85,9,200,0
2,중 구,"2,447",16,114,87,77,236,240,372,386,155,361,403
3,용산구,"2,611",34,71,234,125,221,298,351,125,307,617,228
4,성동구,"3,829",163,144,208,107,325,255,967,415,490,472,283


- 1000 단위 구분기호 없애고 정수로 변환

In [23]:
# cctv.apply(pd.to_numeric, errors='coerce')
for col in cctv.columns[1:]:
    cctv[col] = cctv[col].str.replace(',','').astype(int)
    # cctv[col] = cctv[col].apply(lambda x: int(x.replace(',','')))
cctv.head()

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,83557,4812,1851,3434,4295,6840,8708,11572,10627,12267,11247,7904
1,종로구,1715,815,0,0,195,150,0,261,85,9,200,0
2,중 구,2447,16,114,87,77,236,240,372,386,155,361,403
3,용산구,2611,34,71,234,125,221,298,351,125,307,617,228
4,성동구,3829,163,144,208,107,325,255,967,415,490,472,283


- 최근 3개년 CCTV 증가율 계산 (2021년 제외)

In [26]:
# 최근증가율 = (2018년도 + 2019년도 + 2020년도) / 2017년이전치 합 * 100
# cctv['최근증가율'] = (cctv['2018년'] + cctv['2019년'] + cctv['2020년']) / (cctv['2012년 이전'] + cctv['2012년'] + cctv['2013년'] + cctv['2014년'] + cctv['2015년'] + cctv['2016년'] + cctv['2017년']) * 100
cctv['2017년이전2'] = cctv.iloc[:, 2:9].sum(axis=1)
cctv['최근증가율'] = ((cctv['2018년'] + cctv['2019년'] + cctv['2020년']) / (cctv['2017년이전2']) * 100).round(2)
cctv.head()

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2017년이전2,최근증가율
0,계,83557,4812,1851,3434,4295,6840,8708,11572,10627,12267,11247,7904,41512,82.24
1,종로구,1715,815,0,0,195,150,0,261,85,9,200,0,1421,20.69
2,중 구,2447,16,114,87,77,236,240,372,386,155,361,403,1142,78.98
3,용산구,2611,34,71,234,125,221,298,351,125,307,617,228,1334,78.64
4,성동구,3829,163,144,208,107,325,255,967,415,490,472,283,2169,63.49


- 구분, 총계, 최근증가율 컬럼만 추출하고 컬럼명을 구별, CCTV댓수로 변경하기

In [27]:
cctv2 = cctv[['구분','총계','최근증가율']]
# cctv2.columns = ['구별','CCTV댓수','최근증가율']
cctv2.rename(columns={'구분':'구별', '총계':'CCTV댓수'}, inplace=True)
cctv2

,구별,CCTV댓수,최근증가율
0,계,83557,82.24
1,종로구,1715,20.69
2,중 구,2447,78.98
3,용산구,2611,78.64
4,성동구,3829,63.49
5,광진구,3211,112.29
6,동대문구,2628,36.33
7,중랑구,3737,190.55
8,성북구,4602,77.58
9,강북구,3090,429.29


- 구 이름에서 공백 지우기

In [28]:
cctv2['구별'] = cctv2['구별'].str.replace(' ','')
cctv2

,구별,CCTV댓수,최근증가율
0,계,83557,82.24
1,종로구,1715,20.69
2,중구,2447,78.98
3,용산구,2611,78.64
4,성동구,3829,63.49
5,광진구,3211,112.29
6,동대문구,2628,36.33
7,중랑구,3737,190.55
8,성북구,4602,77.58
9,강북구,3090,429.29


- 첫번째 행(계) 지우기

In [29]:
cctv2.drop([0], inplace=True)
cctv2.head()

,구별,CCTV댓수,최근증가율
1,종로구,1715,20.69
2,중구,2447,78.98
3,용산구,2611,78.64
4,성동구,3829,63.49
5,광진구,3211,112.29


### 인구 데이터 정리하기

- 데이터 파일 읽기

In [30]:
filename2 = '서울시인구.txt'
pop = pd.read_csv(filename2, sep='\t', skiprows=2)
pop.head()

,기간,자치구,세대,계,남자,여자,계.1,남자.1,여자.1,계.2,남자.2,여자.2,세대당인구,65세이상고령자
0,2021,합계,"4,426,007","9,736,027","4,721,977","5,014,050","9,509,458","4,618,040","4,891,418","226,569","103,937","122,632",2.15,"1,605,416"
1,2021,종로구,"73,494","153,789","74,186","79,603","144,683","70,183","74,500","9,106","4,003","5,103",1.97,"27,818"
2,2021,중구,"63,519","131,787","64,083","67,704","122,499","59,630","62,869","9,288","4,453","4,835",1.93,"24,392"
3,2021,용산구,"111,036","237,285","115,085","122,200","222,953","107,210","115,743","14,332","7,875","6,457",2.01,"39,070"
4,2021,성동구,"134,233","292,672","142,259","150,413","285,990","139,380","146,610","6,682","2,879","3,803",2.13,"46,380"


- Null data 있는지 확인하기

In [31]:
pop.isna().sum()

기간          0
자치구         0
세대          0
계           0
남자          0
여자          0
계.1         0
남자.1        0
여자.1        0
계.2         0
남자.2        0
여자.2        0
세대당인구       0
65세이상고령자    0
dtype: int64

- '자치구', '계', '계.1', '계.2', '65세이상고령자‘ 컬럼만 추출하고 이름을 '구별','인구수', '내국인', '외국인', '고령자'로 변경

In [32]:
pop2 = pop[['자치구','계','계.1','계.2','65세이상고령자']]
pop2.columns = ['구별','인구수','내국인','외국인','고령자']
pop2.head()

,구별,인구수,내국인,외국인,고령자
0,합계,"9,736,027","9,509,458","226,569","1,605,416"
1,종로구,"153,789","144,683","9,106","27,818"
2,중구,"131,787","122,499","9,288","24,392"
3,용산구,"237,285","222,953","14,332","39,070"
4,성동구,"292,672","285,990","6,682","46,380"


- 1000 단위 구분기호 없애고 정수로 변환

In [33]:
# pop2.apply(pd.to_numeric, errors='coerce')
for col in pop2.columns[1:]:
    pop2[col] = pop2[col].str.replace(',','').astype(int)
    # pop2[col] = pop2[col].apply(lambda x: int(x.replace(',','')))
pop2.head()

,구별,인구수,내국인,외국인,고령자
0,합계,9736027,9509458,226569,1605416
1,종로구,153789,144683,9106,27818
2,중구,131787,122499,9288,24392
3,용산구,237285,222953,14332,39070
4,성동구,292672,285990,6682,46380


- 첫번째 행 지우기

In [34]:
pop2.drop([0], inplace=True)
pop2.head()

,구별,인구수,내국인,외국인,고령자
1,종로구,153789,144683,9106,27818
2,중구,131787,122499,9288,24392
3,용산구,237285,222953,14332,39070
4,성동구,292672,285990,6682,46380
5,광진구,352627,339996,12631,51723


- '외국인비율‘과'고령자비율‘ 컬럼 만들기

In [35]:
pop2['외국인비율'] = (pop2['외국인'] / pop2['인구수'] * 100).round(2)
pop2['고령자비율'] = np.round(pop2['고령자'] / pop2['인구수'] * 100, 2)
pop2.head()

,구별,인구수,내국인,외국인,고령자,외국인비율,고령자비율
1,종로구,153789,144683,9106,27818,5.92,18.09
2,중구,131787,122499,9288,24392,7.05,18.51
3,용산구,237285,222953,14332,39070,6.04,16.47
4,성동구,292672,285990,6682,46380,2.28,15.85
5,광진구,352627,339996,12631,51723,3.58,14.67


- 구 이름에서 공백 지우기

In [37]:
pop2['구별'] = pop2['구별'].str.replace(' ','')
pop2

,구별,인구수,내국인,외국인,고령자,외국인비율,고령자비율
1,종로구,153789,144683,9106,27818,5.92,18.09
2,중구,131787,122499,9288,24392,7.05,18.51
3,용산구,237285,222953,14332,39070,6.04,16.47
4,성동구,292672,285990,6682,46380,2.28,15.85
5,광진구,352627,339996,12631,51723,3.58,14.67
6,동대문구,352006,337400,14606,62211,4.15,17.67
7,중랑구,391885,387350,4535,71682,1.16,18.29
8,성북구,440142,430528,9614,74709,2.18,16.97
9,강북구,302563,299182,3381,64333,1.12,21.26
10,도봉구,319373,317366,2007,64160,0.63,20.09


### CCTV 데이터와 인구데이터 합치고 저장하기

- 데이터 합치기

In [38]:
cctv_pop = pd.merge(cctv2, pop2)
cctv_pop.head()

,구별,CCTV댓수,최근증가율,인구수,내국인,외국인,고령자,외국인비율,고령자비율
0,종로구,1715,20.69,153789,144683,9106,27818,5.92,18.09
1,중구,2447,78.98,131787,122499,9288,24392,7.05,18.51
2,용산구,2611,78.64,237285,222953,14332,39070,6.04,16.47
3,성동구,3829,63.49,292672,285990,6682,46380,2.28,15.85
4,광진구,3211,112.29,352627,339996,12631,51723,3.58,14.67


- ‘구별’ 컬럼을 인덱스로 만들기

In [39]:
cctv_pop.set_index('구별', inplace=True)
cctv_pop.head()

,CCTV댓수,최근증가율,인구수,내국인,외국인,고령자,외국인비율,고령자비율
구별,,,,,,,,
종로구,1715,20.69,153789,144683,9106,27818,5.92,18.09
중구,2447,78.98,131787,122499,9288,24392,7.05,18.51
용산구,2611,78.64,237285,222953,14332,39070,6.04,16.47
성동구,3829,63.49,292672,285990,6682,46380,2.28,15.85
광진구,3211,112.29,352627,339996,12631,51723,3.58,14.67


- 합친 데이터를 cctv_pop.csv로 저장하기

In [40]:
cctv_pop.to_csv('cctv_pop.csv')

In [43]:
df = pd.read_csv('cctv_pop.csv', index_col='구별')
df.head(3)

,CCTV댓수,최근증가율,인구수,내국인,외국인,고령자,외국인비율,고령자비율
구별,,,,,,,,
종로구,1715,20.69,153789,144683,9106,27818,5.92,18.09
중구,2447,78.98,131787,122499,9288,24392,7.05,18.51
용산구,2611,78.64,237285,222953,14332,39070,6.04,16.47
